# Nonlinear Modeling and Generalization

Thus far, we have illustrated linear regression as “drawing a line through through the data”, 
but this was really a function of our input features.

Though it may seem limited, linear regression algorithms are quite powerful when applied 
to non-linear features of the input data